In [1]:
# Load relevant fucntions from Azure ML 
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [2]:
# Load Azure ID from JSON file
import json

azure_id = open('my_id.json', 'r')
mi = json.load(azure_id)
my_id = mi['my_id']

In [4]:
# Create a new Azure workspace
ws = Workspace.create(name="actividad2_re",
                      subscription_id = my_id,
                      resource_group = "actividad2_re",
                      location = "centralindia")

Deploying KeyVault with name actividakeyvaultdb0dea09.
Deploying StorageAccount with name actividastorage607e3b767.
Deploying AppInsights with name actividainsights4f32df88.
Deployed AppInsights with name actividainsights4f32df88. Took 8.2 seconds.
Deployed KeyVault with name actividakeyvaultdb0dea09. Took 25.46 seconds.
Deployed StorageAccount with name actividastorage607e3b767. Took 55.63 seconds.
Deploying Workspace with name actividad2_re.
Deployed Workspace with name actividad2_re. Took 39.31 seconds.


In [ ]:
# If the workspace already exists, load it
ws = Workspace.get(name="actividad2", subscription_id = my_id, resource_group = "actividad2")

In [5]:
# Load regression model to workspace
mname = "model"
registered_model = Model.register(model_path="reglog.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [6]:
# Create a new environment with needed libraries
virtual_env = Environment("env-2-bankruptcy")
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])

In [7]:
# Load scoring script
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )

# Create a new ACI web service configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

# Deploy the model as a web service
service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\soalv\AppData\Local\Temp\ipykernel_51612\693013631.py:11: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [8]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [9]:
# Generate scoring URI in Azure for model evaluation
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri2.json", "w")
file.write(scoreuri)
file.close()